In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from torrequest import TorRequest
from pprint import pprint
import pandas as pd
from retrying import retry
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import numpy as np

In [6]:
test_players = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/test_players.csv')

In [7]:
test_players.head()

,link,player,team
0,/Players/371335/Show/Juan-Orellana,Juan Orellana,San Martin de Tucuman
1,/Players/345352/Show/Nahuel-Menéndez,Nahuel Menéndez,San Martin de Tucuman
2,/Players/42834/Show/Tino-Costa,Tino Costa,San Martin de Tucuman
3,/Players/135235/Show/Matías-García,Matías García,San Martin de Tucuman
4,/Players/129130/Show/Rodrigo-Gómez,Rodrigo Gómez,San Martin de Tucuman


In [8]:
def get_soup(URL):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

**Whoscored scrapping**

In [9]:
def extract_player_info_block(players_df):
    
    links = players_df.link
    teams = players_df.team
    
    player_info = {'name': [],
                  'position': [],
                  'nationality': [],
                  'height': [],
                  'weight': [],
                  'team': []} 

    for each_link, each_team in zip(links,teams):
        URL = 'https://www.whoscored.com' + each_link
        print(URL)
        soup = get_soup(URL)
        text = ""
        for each in soup.find_all('dl', attrs={'class':'player-info-block'}):
            try:
                text = text + str(each)
            except:
                None
        
        name = re.findall(r'((?<=Name:</dt>\n<dd>).+(?=</dd>\n</dl><dl class))', text)
        height = re.findall(r'((?<=Height:</dt>\n<dd>).+(?=cm</dd>))', text)
        position = re.findall(r'((?<=Positions:</dt>\n<dd>\n<ul>\n<li>).*(?= |</li>))', text)
        nation = re.findall(r'((?<="iconize iconize-icon-left">\n).+(?=\n))', text)
        weight = re.findall(r'((?<=Weight:</dt>\n<dd>).+(?=kg))', text)
        
        player_info['name'].append(name[0]) if len(name) >0 else player_info['name'].append(np.nan)
        player_info['position'].append(position[0].strip()) if len(position) >0 else player_info['position'].append(np.nan)
        player_info['nationality'].append(nation[0].strip()[0:3].upper()) if len(nation) >0 else player_info['nationality'].append(np.nan)
        player_info['height'].append(height[0]) if len(height) >0 else player_info['height'].append(np.nan)
        player_info['weight'].append(weight[0]) if len(weight) >0 else player_info['weight'].append(np.nan)
        player_info['team'].append(each_team)

    player_info_df = pd.DataFrame(player_info)
    
    return player_info_df

In [10]:
test = extract_player_info_block(test_players.iloc[0:5,:])

https://www.whoscored.com/Players/371335/Show/Juan-Orellana

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/345352/Show/Nahuel-Menéndez

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/42834/Show/Tino-Costa

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/135235/Show/Matías-García

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
https://www.whoscored.com/Players/129130/Show/Rodrigo-Gómez

Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/gonzaloferreiro/.wdm/chromedriver/74.0.3729.6/mac64/chrom

In [12]:
test

,name,position,nationality,height,weight,team
0,Juan Orellana,Defender,NaN,193,NaN,San Martin de Tucuman
1,Nahuel Menéndez,Defender (Right),ARG,183,72,San Martin de Tucuman
2,Tino Costa,Midfielder (Centre),ARG,177,74,San Martin de Tucuman
3,Matías García,Defender (Left),NaN,174,71,San Martin de Tucuman
4,Rodrigo Gómez,"Attacking Midfielder (Centre, Right)",ARG,168,65,San Martin de Tucuman


In [261]:
test_text = test

In [310]:
test_text

'<dl class="player-info-block">\n<dt>Name:</dt>\n<dd>Nahuel Menéndez</dd>\n</dl><dl class="player-info-block">\n<dt>Current Team:</dt>\n<dd>\n<a class="team-link" href="/Teams/5595/Show/Argentina-San-Martin-de-Tucuman">San Martin de Tucuman</a>\n</dd>\n</dl><dl class="player-info-block">\n<dt>Shirt Number:</dt>\n<dd>24</dd>\n</dl><dl class="player-info-block">\n<dt>Positions:</dt>\n<dd>\n<ul>\n<li>Defender (Right) </li>\n<li> Midfielder (Right)</li>\n</ul>\n</dd>\n</dl><dl class="player-info-block">\n<dt>Age:</dt>\n<dd>25 years old (<i>05-03-1994</i>)</dd>\n</dl><dl class="player-info-block">\n<dt>Height:</dt>\n<dd>183cm</dd>\n</dl><dl class="player-info-block">\n<dt>Weight:</dt>\n<dd>72kg</dd>\n</dl><dl class="player-info-block">\n<dt>Nationality:</dt>\n<dd>\n<span class="iconize iconize-icon-left">\n                                Argentina\n                                <span class="ui-icon country flg-ar"></span>\n</span>\n</dd>\n</dl>'

In [193]:
re.findall(r'((?<=Name:</dt>\n<dd>).+(?=</dd>\n</dl><dl class))', test_text)

['Juan Orellana']

In [179]:
re.findall(r'((?<=Height:</dt>\n<dd>).+(?=cm</dd>))', test_text)

['193']

In [323]:
re.findall(r'((?<=Positions:</dt>\n<dd>\n<ul>\n<li>).*(?= |</li>))', test_text)

['Defender (Right) ']

In [140]:
position

'Defender'

In [143]:
nation = re.findall(r'((?<="iconize iconize-icon-left">\n).+(?=\n)|(?=\n))', test_text)[0].strip()[0:3].upper()

In [144]:
nation

'ARG'

In [87]:
weight = re.findall(r'((?<=Weight:</dt>\n<dd>).+(?=kg))', test_text)[0]